In [1]:
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout, Activation
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping
from sklearn import model_selection
from sklearn import metrics
import time
import tensorflow as tf
import shap
import numpy as np

C:\Users\Ameya\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
df = pd.read_csv('UNSW_NB15_balanced_smote_lime.csv')
columns = df.columns.tolist()

In [3]:
#read kdd_processed_bin_top15.csv
df_bin = pd.read_csv('unsw_shap_top_15.csv')

# get the names of the columns in a list
columns_bin = df_bin.columns.tolist()

In [4]:
#combine the two lists of columns without duplicates
columns = list(set(columns + columns_bin))
print(columns)
#print length of columns
print(len(columns))

['rate', 'proto_bbn-rcc', 'state_CLO', 'proto_mhrp', 'sttl', 'proto_narp', 'ackdat', 'proto_wsn', 'dmean', 'proto_udp', 'proto_ipnip', 'proto_isis', 'proto_tcf', 'ct_dst_src_ltm', 'ct_state_ttl', 'service_smtp', 'proto_aris', 'proto_tlsp', 'is_sm_ips_ports', 'dload', 'proto_igp', 'proto_pri-enc', 'proto_ipcomp', 'dwin', 'ct_srv_dst', 'dttl', 'proto_wb-expak', 'sload', 'proto_pvp', 'swin', 'label']
31


In [6]:
data = pd.read_csv('UNSW_NB15_balanced_smote.csv')

In [7]:
data = data.drop(data.columns.difference(columns), axis=1)
data

,rate,sttl,dttl,sload,dload,swin,dwin,ackdat,dmean,ct_state_ttl,...,proto_narp,proto_pri-enc,proto_pvp,proto_tcf,proto_tlsp,proto_udp,proto_wb-expak,proto_wsn,service_smtp,state_CLO
0,0.575396,2.440529,0.000000,0.976642,0.000000,0.000000,0.000000,0.000000,0.000000,1.996613,...,False,False,False,False,False,True,False,False,False,False
1,0.791170,2.440529,0.000000,4.770483,0.000000,0.000000,0.000000,0.000000,0.000000,1.996613,...,False,False,False,False,False,True,False,False,False,False
2,1.265872,2.440529,0.000000,4.626448,0.000000,0.000000,0.000000,0.000000,0.000000,1.996613,...,False,False,False,False,False,True,False,False,False,False
3,1.054893,2.440529,0.000000,3.248910,0.000000,0.000000,0.000000,0.000000,0.000000,1.996613,...,False,False,False,False,False,True,False,False,False,False
4,0.632936,2.440529,0.000000,4.604789,0.000000,0.000000,0.000000,0.000000,0.000000,1.996613,...,False,False,False,False,False,True,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
557995,0.000157,2.440529,2.227963,0.000032,0.001442,2.000058,2.000571,0.370652,0.169675,0.998307,...,False,False,False,False,False,False,False,False,False,False
557996,0.779553,2.440529,0.000000,0.448168,0.000000,0.000000,0.000000,0.000000,0.000000,1.996613,...,False,False,False,False,False,True,False,False,False,False
557997,0.000080,2.440529,2.227963,0.000016,0.000738,2.000058,2.000571,1.596556,0.169675,0.998307,...,False,False,False,False,False,False,False,False,False,False
557998,0.703262,2.440529,0.000000,0.404309,0.000000,0.000000,0.000000,0.000000,0.000000,1.996613,...,False,False,False,False,False,True,False,False,False,False


In [ ]:
# train an lstm model on the data
X = data.drop(['label'], axis=1)
Y = data['label']
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.2, random_state = 42)

# convert Y to int
Y_train = Y_train.astype(int)
Y_test = Y_test.astype(int)

# reshape the data for LSTM
X_train = X_train.values.reshape((X_train.shape[0], 1, X_train.shape[1]))
X_test = X_test.values.reshape((X_test.shape[0], 1, X_test.shape[1]))

# convert all to numpy arrays
X_train = np.asarray(X_train).astype(np.float32)
X_test = np.asarray(X_test).astype(np.float32)

# define the model and add recurrent dropout
model = Sequential()
model.add(LSTM(64, input_shape=(X_train.shape[1], X_train.shape[2]), return_sequences=True, activation = 'relu'))
model.add(Dropout(0.1))
model.add(LSTM(64, return_sequences=True, activation = 'relu'))
model.add(Dropout(0.1))
model.add(LSTM(64, activation = 'relu'))
model.add(Dropout(0.1))
model.add(Dense(1, activation='sigmoid'))

# compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# train the model
start = time.time()
history = model.fit(X_train, Y_train, epochs=10, batch_size=32, validation_data=(X_test, Y_test), shuffle=True)
end = time.time()
print("Time taken to train the model: ", end-start)

Epoch 1/10
23250/23250 [==============================] - 36s 2ms/step - loss: 0.0929 - accuracy: 0.9702 - val_loss: 0.0837 - val_accuracy: 0.9704
Epoch 2/10
23250/23250 [==============================] - 35s 2ms/step - loss: 0.0834 - accuracy: 0.9708 - val_loss: 0.0842 - val_accuracy: 0.9701
Epoch 3/10
23250/23250 [==============================] - 35s 2ms/step - loss: 0.0810 - accuracy: 0.9709 - val_loss: 0.0810 - val_accuracy: 0.9703
Epoch 4/10
23250/23250 [==============================] - 36s 2ms/step - loss: 0.0799 - accuracy: 0.9709 - val_loss: 0.0805 - val_accuracy: 0.9704
Epoch 5/10
23250/23250 [==============================] - 35s 2ms/step - loss: 0.0792 - accuracy: 0.9709 - val_loss: 0.0839 - val_accuracy: 0.9704
Epoch 6/10
23250/23250 [==============================] - 36s 2ms/step - loss: 0.0789 - accuracy: 0.9710 - val_loss: 0.0773 - val_accuracy: 0.9706
Epoch 7/10
23250/23250 [==============================] - 36s 2ms/step - loss: 0.0785 - accuracy: 0.9710 - val_loss: 0

In [8]:
from sklearn.model_selection import KFold

kfold = KFold(n_splits=5, shuffle=True)
X = data.drop(['label'], axis=1)
Y = data['label']

# reshape the data for LSTM
X_fit = X.values.reshape((X.shape[0], 1, X.shape[1]))
X_fit = np.asarray(X_fit).astype(np.float32)

# define the model
model = Sequential()
model.add(LSTM(64, input_shape=(X_fit.shape[1], X_fit.shape[2]), return_sequences=True, activation = 'relu'))
model.add(Dropout(0.1))
model.add(LSTM(64, return_sequences=True, activation = 'relu'))
model.add(Dropout(0.1))
model.add(LSTM(64, activation = 'relu'))
model.add(Dropout(0.1))
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


accuracies = []

for train_idx, test_idx in kfold.split(X_fit):
    model.fit(X_fit[train_idx], Y[train_idx], epochs=10, batch_size=32, verbose=0)
    accuracy = model.evaluate(X_fit[test_idx], Y[test_idx], verbose=0)[1] 
    accuracies.append(accuracy)
    print(accuracy)

print("\n\n\n final- ",accuracies)
#mean accuracy
print("Mean- ",np.mean(accuracies))

0.9515681266784668
0.9525896310806274
0.9523118138313293
0.9518907070159912
0.952804684638977



 final-  [0.9515681266784668, 0.9525896310806274, 0.9523118138313293, 0.9518907070159912, 0.952804684638977]
Mean-  0.9522329926490783


In [8]:
start_time = time.time()
y_pred = model.predict(X_test)
end_time = time.time()
y_pred = (y_pred > 0.5)
print('Classification Report')
print(metrics.classification_report(Y_test, y_pred, digits=4))
print('Accuracy:')
print(metrics.accuracy_score(Y_test, y_pred))
print("Time taken to predict: ", end_time-start_time)

5813/5813 [==============================] - 3s 526us/step
Classification Report
              precision    recall  f1-score   support

           0     0.9952    0.7123    0.8303     18731
           1     0.9688    0.9996    0.9840    167269

    accuracy                         0.9707    186000
   macro avg     0.9820    0.8560    0.9071    186000
weighted avg     0.9714    0.9707    0.9685    186000

Accuracy:
0.9706774193548388
Time taken to predict:  4.097585916519165
